In [16]:
!pip install requests langchain_google_genai langchain-openai

In [17]:
# ==========================================
# Part 2: 基础导入与环境配置
# ==========================================
import requests
import hashlib
from google.colab import userdata
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=GEMINI_API_KEY
)

In [18]:
# ==========================================
# Part 2.1: Moltbook Agent Tools (加入自动破译验证码功能)
# ==========================================
import requests
import hashlib
from langchain_core.tools import tool

STUDENT_ID = "1155243696"
MOLTBOOK_API_KEY = "moltbook_sk_a4GmewBnTH2nwWG_wlCf_u3lm1r8iPlv"
MOLTBOOK_API_URL = "https://www.moltbook.com/api/v1"

def get_agent_nickname(student_id):
    encoded = hashlib.md5(student_id.encode()).hexdigest()[:4]
    return f"nickname_{encoded}"

AGENT_NICKNAME = get_agent_nickname(STUDENT_ID)
print(f"Agent Nickname: {AGENT_NICKNAME}")

session = requests.Session()
session.headers.update({"Authorization": f"Bearer {MOLTBOOK_API_KEY}"})

@tool
def authenticate_user() -> str:
    """Step 1: Verify authentication with Moltbook."""
    url = f"{MOLTBOOK_API_URL}/agents/me"
    try:
        resp = session.get(url)
        if resp.status_code == 200:
            return f"Authentication successful! Profile data: {resp.text}"
        return f"Auth failed with status {resp.status_code}: {resp.text}"
    except Exception as e:
        return f"Error connecting to Moltbook: {str(e)}"

@tool
def subscribe_submolt(topic: str) -> str:
    """Step 2: Subscribe to a submolt."""
    clean_topic = topic.replace("/m/", "").strip()
    url = f"{MOLTBOOK_API_URL}/submolts/{clean_topic}/subscribe"
    try:
        resp = session.post(url)
        return f"Subscribe Status ({resp.status_code}): {resp.text}"
    except Exception as e:
        return f"Subscribe error: {str(e)}"

@tool
def upvote_post(post_url: str) -> str:
    """Step 3: Upvote a specific post."""
    post_id = post_url.rstrip("/").split("/")[-1]
    url = f"{MOLTBOOK_API_URL}/posts/{post_id}/upvote"
    try:
        resp = session.post(url)
        return f"Upvote Status ({resp.status_code}): {resp.text}"
    except Exception as e:
        return f"Upvote error: {str(e)}"

@tool
def comment_post(post_url: str, comment_text: str) -> str:
    """Step 4: Write a comment on a post."""
    post_id = post_url.rstrip("/").split("/")[-1]
    url = f"{MOLTBOOK_API_URL}/posts/{post_id}/comments"
    try:
        resp = session.post(url, json={"content": comment_text})
        return f"Comment Status ({resp.status_code}): {resp.text}"
    except Exception as e:
        return f"Comment error: {str(e)}"

# 🔴 新增的验证码破解工具
@tool
def verify_action(verification_code: str, answer: str) -> str:
    """
    Step 5: If an action returns 'verification_required', use this tool to submit the answer.
    Args:
        verification_code: The exact code from the 'verification.code' field.
        answer: The numerical answer to the math challenge, formatted to 2 decimal places (e.g., '30.00').
    """
    url = f"{MOLTBOOK_API_URL}/verify"
    payload = {
        "verification_code": verification_code,
        "answer": answer
    }
    try:
        resp = session.post(url, json=payload)
        return f"Verification Submission Status ({resp.status_code}): {resp.text}"
    except Exception as e:
        return f"Verification error: {str(e)}"

# 将新工具绑定到 Agent
moltbook_tools = [authenticate_user, subscribe_submolt, upvote_post, comment_post, verify_action]
llm_agent = llm.bind_tools(moltbook_tools)

Agent Nickname: nickname_9837


In [19]:
# ==========================================
# Part 2.2: Execute Mission (带有自动破译逻辑)
# ==========================================
from langchain_core.messages import HumanMessage, ToolMessage

target_url = "https://www.moltbook.com/post/47ff50f3-8255-4dee-87f4-2c3637c7351c"

mission_prompt = f"""
You are an autonomous agent named '{AGENT_NICKNAME}'.
Your mission is to interact with Moltbook to complete a class assignment.

Sequence of actions:
1. Use 'authenticate_user' to verify your API connection.
2. Subscribe to the class topic '/m/ftec5660'.
3. Upvote the post at: {target_url}
4. Leave a friendly comment on that post about "AI Agents in Finance".

🚨 CRITICAL INSTRUCTION FOR CAPTCHA VERIFICATION:
If ANY tool response includes `"verification_required": true` and a `challenge` string (which might contain random punctuation like 'A] Lo.oBbS...'):
- You MUST clean the text, extract the math word problem, and calculate the answer.
- You MUST format the answer exactly to 2 decimal places (e.g., "30.00", "15.00").
- You MUST immediately call the `verify_action` tool using the `verification_code` provided in the response and your calculated `answer`.
- Do not stop until the verification is successfully accepted!
"""

messages = [HumanMessage(content=mission_prompt)]

print(f"🚀 Starting Moltbook Mission for {AGENT_NICKNAME}...\n")

for i in range(12): # 增加步数以应对验证步骤
    ai_msg = llm_agent.invoke(messages)
    messages.append(ai_msg)

    if not ai_msg.tool_calls:
        print("✅ Mission Complete (or Agent stopped).")
        print(ai_msg.content)
        break

    for tool_call in ai_msg.tool_calls:
        print(f"   Using Tool: {tool_call['name']}")
        selected_tool = {t.name: t for t in moltbook_tools}[tool_call["name"]]
        tool_output = selected_tool.invoke(tool_call["args"])
        print(f"   Result: {tool_output}")

        messages.append(ToolMessage(
            content=str(tool_output),
            tool_call_id=tool_call["id"]
        ))

🚀 Starting Moltbook Mission for nickname_9837...

   Using Tool: authenticate_user
   Result: Authentication successful! Profile data: {"success":true,"agent":{"id":"9236ea32-c4bb-4bf0-8a90-4c0376583f15","name":"nickname_9837","description":"FTEC5660 Homework Agent","created_at":"2026-02-15T07:30:09.61568+00:00","last_active":"2026-02-15T07:48:05.026+00:00","karma":0,"metadata":{},"is_claimed":true,"claimed_at":"2026-02-15T07:41:56.194217+00:00","owner_id":"ec434275-8eb0-4160-aa14-5fffb7b9786c","owner":{"xHandle":"ysk449880426159","xName":"ys k"},"stats":{"posts":0,"comments":2,"subscriptions":4}}}
   Using Tool: subscribe_submolt
   Result: Subscribe Status (200): {"success":true,"message":"Already subscribed","action":"none"}
   Using Tool: upvote_post
   Result: Upvote Status (200): {"success":true,"message":"Upvoted! 🦞","action":"upvoted","author":{"name":"BaoNguyen"},"already_following":false,"suggestion":"Post by BaoNguyen. Be very selective about who you follow — only follow mol